In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
import sys
from os.path import join

sys.path.insert(0, '/'.join(sys.path[0].split('/')[:-1] + ['scripts']))
from optimizer import *

In [3]:
#export
class SubModel(Module):
    def __init__(self):
        super().__init__()
    
    def parameters(self):
        for param in self.sub_model.parameters():
            yield param
            
    def fwd(self, inp):
        return self.sub_model(inp)
    
    def bwd(self, out, inp):
        self.sub_model.backward()
        
    def __repr__(self, t):
        return self.sub_model.__repr__(t+'    ')

## Test

In [4]:
class FirstNest(SubModel):
    def __init__(self):
        self.sub_model = Sequential(Linear(700, 640), ReLU(), SecondNest(), Linear(40, 20), ReLU())

class SecondNest(SubModel):
    def __init__(self):
        self.sub_model = Sequential(Linear(640, 320), ReLU(), ThirdNest(), Linear(80, 40), ReLU())
        
class ThirdNest(SubModel):
    def __init__(self):
        self.sub_model = Sequential(Linear(320, 160), ReLU(), Linear(160, 80), ReLU())

In [5]:
from training import fit
from optimizer import Optimizer

data_bunch = get_data_bunch(*get_mnist_data(), batch_size=64)
model = Sequential(Linear(784, 700), ReLU(), FirstNest(), Linear(20, 10), ReLU())
optimizer = Optimizer(list(model.parameters()), learning_rate=0.1)
loss_fn = CrossEntropy()

model

(Model)
    Linear(784, 700)
    ReLU()
        Linear(700, 640)
        ReLU()
            Linear(640, 320)
            ReLU()
                Linear(320, 160)
                ReLU()
                Linear(160, 80)
                ReLU()
            Linear(80, 40)
            ReLU()
        Linear(40, 20)
        ReLU()
    Linear(20, 10)
    ReLU()

In [6]:
accuracies, losses = fit(1, data_bunch, model, loss_fn, optimizer)

Epoch 1  Accuracy 0.966  Loss 0.121
